In [1]:
import pandas as pd
import os
import time
import subprocess
import json
from collections import defaultdict

TEMP_DIR = 'scratchpad'
PROCESSED_COMMENTS_DIR = 'processed_comments'
COMMENTS_CSV = f'{PROCESSED_COMMENTS_DIR}/comments.csv'

os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(PROCESSED_COMMENTS_DIR, exist_ok=True)

def run_yt_dlp(video_id, max_comments=100):
    filepath = os.path.join(TEMP_DIR, f"{video_id}")
    command = [
        "yt-dlp",
        "--skip-download",
        "--write-comments",
        "--no-warnings",
        "--output", filepath,
        "--extractor-args", f"youtube:max_comments={max_comments}",
        f"https://www.youtube.com/watch?v={video_id}"
    ]
    subprocess.run(command, check=True, text=True)


def parse_comments(video_id):
    with open(os.path.join(TEMP_DIR, f'{video_id}.info.json'), "r", encoding="utf-8") as f:
        data = json.load(f)
    comments = data['comments']
    title = data['title']
    uploader = data['uploader_id']

    comment_dict = {}
    replies = defaultdict(list)
    for c in comments:
        if c.get("parent") and c["parent"] != "root":
            replies[c["parent"]].append({**c, "is_reply": True})
        else:
            comment_dict[c["id"]] = {**c, "replies": [], "is_reply": False}
    for pid, rep in replies.items():
        if pid in comment_dict:
            comment_dict[pid]["replies"] = rep
    return list(comment_dict.values()), title, uploader


def to_process(video_id):
    if not os.path.exists(COMMENTS_CSV):
        return True
    df = pd.read_csv(COMMENTS_CSV)
    return video_id not in df['id'].astype(str).values


# Main loop
df_url_details = pd.read_csv('URLs/url_channel_id.csv')
data_res = []

for video_id in df_url_details.id.astype(str):
    print('going for video_id',video_id)
    if not to_process(video_id):
        print(f"⏩ Skipping already processed video ID: {video_id}")
        continue

    try:
        run_yt_dlp(video_id, max_comments=100)
        comments, title, uploader = parse_comments(video_id)

        data_res.append({
            'creator_id': uploader,
            'id': video_id,
            'title': title,
            'timestamp': str(time.time()),
            'comments': json.dumps(comments)
        })

        # Append to CSV immediately to avoid reprocessing on crash
        df = pd.DataFrame(data_res)
        if os.path.exists(COMMENTS_CSV):
            df.to_csv(COMMENTS_CSV, mode='a', index=False, header=False)
        else:
            df.to_csv(COMMENTS_CSV, index=False)
        data_res.clear()

    except Exception as e:
        print(f"❌ Failed to process video ID: {video_id}")
        print(e)

going for video_id QECHcQyQhN0
[youtube] Extracting URL: https://www.youtube.com/watch?v=QECHcQyQhN0
[youtube] QECHcQyQhN0: Downloading webpage
[youtube] QECHcQyQhN0: Downloading tv client config
[youtube] QECHcQyQhN0: Downloading player 6740c111-main
[youtube] QECHcQyQhN0: Downloading tv player API JSON
[youtube] QECHcQyQhN0: Downloading ios player API JSON
[youtube] QECHcQyQhN0: Downloading m3u8 information
[youtube] Downloading comment section API JSON
[youtube] Downloading ~1161 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/~1161)
[youtube]     Downloading comment API JSON reply thread 1 (16/~1161)
[youtube]        Downloading comment replies API JSON page 1 (26/~1161)
[youtube] Downloading comment API JSON page 2 (44/~1161)
[youtube]     Downloading comment API JSON reply thread 1 (63/~1161)
[youtube] Downloading comment API JSON page 3 (65/~1161)
[youtube]     Downloading comment API JSON reply thread 1 (72/~1161)
[youtube]  

In [ ]:
df_test=pd.read_csv(f'{PROCESSED_COMMENTS_DIR}/comments.csv')
df_test['comments'] = df_test['comments'].apply(json.loads)

In [ ]:
df_test.shape

In [ ]:
df_test.tail()